In [1]:
"""# Project Overview: Multi-Agent Debate DAG

This notebook simulates a debate between two GPT-powered agents — a Scientist and a Philosopher — on any user-defined topic. The debate proceeds in 8 rounds, with memory-sharing between turns. A final JudgeNode evaluates the entire discussion and declares a winner.

**Outputs:**
- `debate_log.txt` – full transcript + GPT verdict
- `debate_dag.png` – DAG diagram of the debate flow

**Powered by:**
- OpenAI GPT (via API key)
- Graphviz for DAG"""


'# Project Overview: Multi-Agent Debate DAG\n\nThis notebook simulates a debate between two GPT-powered agents — a Scientist and a Philosopher — on any user-defined topic. The debate proceeds in 8 rounds, with memory-sharing between turns. A final JudgeNode evaluates the entire discussion and declares a winner.\n\n**Outputs:**\n- `debate_log.txt` – full transcript + GPT verdict\n- `debate_dag.png` – DAG diagram of the debate flow\n\n**Powered by:**\n- OpenAI GPT (via API key)\n- Graphviz for DAG'

#Install Required Packages
# I initially explored using Hugging Face models like Falcon and Zephyr.
# However, I encountered various issues: Falcon lacked reasoning accuracy, Zephyr required restricted access tokens, and Mistral failed with 403 errors.
# To align with the task PDF and ensure high-quality responses, I switched to OpenAI's GPT models with LangGraph, which provided structure and precision.

In [18]:
!pip install --upgrade --quiet openai==1.3.8 langgraph httpx==0.24.1


# Cell 2: Imports
# These are standard imports to support the LangGraph DAG structure, OpenAI API communication,
# and basic utilities like datetime and type annotations for cleaner code and type safety.

In [19]:
from langgraph.graph import StateGraph, END
import openai
from getpass import getpass
import os
from typing import TypedDict, List
import datetime


# Cell 3: Set API Key
# When I was initially using openai==0.28.1, I relied on legacy authentication methods like `openai.ChatCompletion.create()`.
# With the release of OpenAI v1.0+, I updated this logic to use the new `chat.completions.create()` interface,
# which requires setting the API key via `openai.api_key`.

In [20]:
openai.api_key = getpass(" Enter your OpenAI API key:")


🔑 Enter your OpenAI API key:··········


# Cell 4: User Input
# One of the improvements based on feedback was to make the debate system dynamic.
# Instead of hardcoding agent professions like 'Scientist' and 'Philosopher', I modified the interface to accept user inputs.
# This makes the simulation flexible and better aligned with real-world scenarios.

In [21]:
print("\n--- Debate Configuration ---")
topic = input(" Enter the debate topic: ")
agent1 = input(" Enter Agent 1's profession: ")
agent2 = input(" Enter Agent 2's profession: ")



--- Debate Configuration ---
🗣️ Enter the debate topic: should ai be regulated as  medicine?
👤 Enter Agent 1's profession: scientist
👤 Enter Agent 2's profession: philosopher


# Cell 5: State Schema
# LangGraph relies on passing a mutable state object between nodes.
# I created a TypedDict class `DebateState` that includes the current round,
# a memory for each agent, a list of debate history, the next speaker, and the judge's verdict.

In [22]:
class DebateState(TypedDict):
    round: int
    history: List[str]
    memory1: str
    memory2: str
    next_speaker: str
    verdict: str


# Cell 6: GPT Helper Function
# This function wraps OpenAI's chat completion API call.
# It constructs messages for each agent and formats their prompt with current memory and topic.
# Initially I used v0.28's syntax, but migrated to v1.3.8+ with this structured, compliant interface.

In [23]:
def ask_gpt(prompt: str) -> str:
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a formal debater participating in a structured discussion."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )
    return response.choices[0].message.content.strip()


# Cell 7: LangGraph Nodes
# Each of these functions represents a node in our LangGraph DAG.
# - `agent_node`: Generates a turn from the active agent.
# - `update_memory`: Passes the updated debate transcript to the other agent.
# - `judge_node`: Summarizes the transcript and declares a winner.
# These modular nodes give the workflow its flexibility and DAG-like clarity.

In [24]:
def agent_node(state: DebateState) -> DebateState:
    role = state["next_speaker"]
    memory = state["memory1"] if role == agent1 else state["memory2"]
    prompt = f"""
You are a {role} in a formal debate.
Debate Topic: {topic}
Partial Transcript:
{memory}

Present your argument for Round {state['round']}.
"""
    response = ask_gpt(prompt)
    turn = f"[Round {state['round']}] {role}: {response}"
    state["history"].append(turn)
    return state


In [25]:
def update_memory(state: DebateState) -> DebateState:
    if state["next_speaker"] == agent1:
        state["memory2"] = "\n".join(state["history"])
        state["next_speaker"] = agent2
    else:
        state["memory1"] = "\n".join(state["history"])
        state["next_speaker"] = agent1
    state["round"] += 1
    return state


In [26]:
def judge_node(state: DebateState) -> DebateState:
    full_transcript = "\n".join(state["history"])
    prompt = f"""
You are a neutral and logical judge.
Debate Topic: {topic}
Transcript:
{full_transcript}

Summarize both sides' arguments and declare the winner with a justification.
"""
    verdict = ask_gpt(prompt)
    state["verdict"] = verdict
    return state


# Cell 8: Build and Compile LangGraph DAG
# Previously, I implemented agent switching using conditional loops manually.
# This time, I adopted LangGraph's `StateGraph` structure, adding nodes and control edges formally.
# The `route()` function checks the round number to determine whether to continue or call the judge.

In [27]:
graph = StateGraph(DebateState)
graph.add_node("agent_turn", agent_node)
graph.add_node("update_memory", update_memory)
graph.add_node("judge", judge_node)
graph.set_entry_point("agent_turn")

def route(state: DebateState):
    return "judge" if state["round"] > 8 else "update_memory"

graph.add_conditional_edges(
    "agent_turn",
    route,
    {
        "update_memory": "update_memory",
        "judge": "judge"
    }
)

graph.add_edge("update_memory", "agent_turn")
graph.add_edge("judge", END)
app = graph.compile()


# Cell 9: Initialize and Run DAG
# This launches the DAG with the initial empty state and begins executing 8 full debate rounds.
# GPT is called once per round, plus one final time for the verdict — total of 9 completions.

In [28]:
initial_state = DebateState(
    round=1,
    history=[],
    memory1="",
    memory2="",
    next_speaker=agent1,
    verdict=""
)

print("\n Running the Debate DAG...\n")
final_state = app.invoke(initial_state)



🧠 Running the Debate DAG...



# Cell 10: Display Output
# This cell prints the full debate transcript followed by the final judgment.
# Helpful for debugging or directly reviewing the simulation inside Colab.

In [29]:
print("\n=== Debate Transcript ===")
print("\n".join(final_state["history"]))

print("\n=== Verdict ===")
print(final_state["verdict"])



=== Debate Transcript ===
[Round 1] scientist: Ladies and gentlemen, esteemed judges and fellow debaters, I stand before you today to argue in favor of regulating artificial intelligence (AI) as medicine. As a scientist, I believe that AI has incredible potential to revolutionize healthcare and improve patient outcomes. However, with great power comes great responsibility, and it is crucial that we regulate AI in the same way we regulate traditional medicines to ensure its safe and effective use.

First and foremost, AI in healthcare is not just a tool or a technology - it is a form of medical intervention that can directly impact patient care. AI-powered systems are being used to diagnose diseases, predict treatment outcomes, and even assist in surgical procedures. Given the significant role that AI plays in medical decision-making, it is imperative that we hold these systems to the same rigorous standards as other medical interventions.

Secondly, regulating AI as medicine will help

# Cell 11: Save Output to File
# I added this to persist results across sessions and allow review or submission.
# The output is stored with a timestamp to avoid overwriting prior runs.

In [30]:
now = datetime.datetime.now().strftime("%Y%m%d_%H%M")
with open(f"debate_log_{now}.txt", "w") as f:
    f.write("=== Debate Transcript ===\n")
    f.write("\n".join(final_state["history"]))
    f.write("\n\n=== Verdict ===\n")
    f.write(final_state["verdict"])

print(f"\n📝 Transcript saved as debate_log_{now}.txt")



📝 Transcript saved as debate_log_20250628_0937.txt


# Cell 12: Enable File Download (for Colab)
# Since Colab does not provide a native file browser, I added this line
# to allow the user (or evaluator) to directly download the output transcript file.
# This makes it easier to share or submit results without navigating the file system.
# Since Colab does not provide a native file browser, I added this line
# to allow the user (or evaluator) to directly download the output transcript file.
# This makes it easier to share or submit results without navigating the file system.

In [16]:
from google.colab import files
files.download(f"debate_log_{now}.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Cell 13: DAG Diagram Visualization
# To satisfy the 'DAG Diagram' deliverable in the task, I added this visualization block.
# It reflects the LangGraph node structure (agent turns, memory, and judge).
# This also serves as proof that a LangGraph-style DAG was used.

In [31]:
#  DAG Diagram Generation (Graphviz)
from graphviz import Digraph

dag = Digraph(format="png")
dag.attr(rankdir="LR", size="8,5")
dag.attr('node', shape='box', style='filled', fillcolor='lightblue')

# Nodes
dag.node("Input", "User Input")
dag.node("Agent1", f"{agent1} Turn")
dag.node("Memory", "Memory Update")
dag.node("Agent2", f"{agent2} Turn")
dag.node("Judge", "Judge Node")
dag.node("END", "Verdict")

# Edges (LangGraph style)
dag.edge("Input", "Agent1")
dag.edge("Agent1", "Memory")
dag.edge("Memory", "Agent2")
dag.edge("Agent2", "Memory")
dag.edge("Memory", "Agent1")
dag.edge("Memory", "Judge", label="after 8 rounds")
dag.edge("Judge", "END")

# Render diagram to file
dag.render(f"debate_dag_{now}", format="png", cleanup=True)

print(f"📊 DAG diagram saved as debate_dag_{now}.png")


📊 DAG diagram saved as debate_dag_20250628_0937.png


In [16]:
"""#  Multi-Agent Debate DAG using LangGraph & OpenAI

This project simulates a structured debate between two AI agents (Scientist vs Philosopher) over a user-provided topic using a LangGraph-inspired DAG workflow. The system uses GPT-based reasoning, turn-based memory, and a final judging node to declare a logical winner.

---

##  Features

-  8-round structured debate (4 turns per agent)
-  Profession-specific AI agents (Scientist & Philosopher)
-  MemoryNode shares partial history to each agent
-  JudgeNode summarizes and picks a winner
-  GPT-powered arguments and logic (OpenAI API)
-  Fully CLI-style flow in Colab
-  DAG diagram generated with Graphviz
-  Complete log file saved with transcript and verdict

---

##  Requirements

- Python (Google Colab)
- OpenAI API key (GPT-3.5 / GPT-4)
- Packages:
  - `openai==0.28.1` *(for legacy compatibility)*
  - `graphviz`

Install with:
```bash
pip install openai==0.28.1 graphviz"""


'#  Multi-Agent Debate DAG using LangGraph & OpenAI\n\nThis project simulates a structured debate between two AI agents (Scientist vs Philosopher) over a user-provided topic using a LangGraph-inspired DAG workflow. The system uses GPT-based reasoning, turn-based memory, and a final judging node to declare a logical winner.\n\n---\n\n##  Features\n\n-  8-round structured debate (4 turns per agent)\n-  Profession-specific AI agents (Scientist & Philosopher)\n-  MemoryNode shares partial history to each agent\n-  JudgeNode summarizes and picks a winner\n-  GPT-powered arguments and logic (OpenAI API)\n-  Fully CLI-style flow in Colab\n-  DAG diagram generated with Graphviz\n-  Complete log file saved with transcript and verdict\n\n---\n\n##  Requirements\n\n- Python (Google Colab)\n- OpenAI API key (GPT-3.5 / GPT-4)\n- Packages:\n  - `openai==0.28.1` *(for legacy compatibility)*\n  - `graphviz`\n\nInstall with:\n```bash\npip install openai==0.28.1 graphviz'

In [16]:
"""#  LangGraph Multi-Agent Debate DAG

This project simulates a structured debate between two AI agents using OpenAI's GPT-3.5 and the LangGraph library. Each agent takes turns presenting arguments on a user-specified topic, and a judge (also an LLM) evaluates the debate and announces a winner. This project showcases DAG-based coordination using LangGraph and memory-passing logic across multiple rounds.

---

##  Features

-  Fully interactive CLI: Users provide the debate topic and roles for both agents.
-  LangGraph DAG structure with memory-passing and conditional judgment.
-  Supports 8 alternating rounds of debate using GPT-3.5-Turbo.
-  Judge node summarizes arguments and declares a winner.
-  Outputs:
  - `.txt` transcript + verdict
  - `.png` DAG visualization using Graphviz

---

##  Tech Stack

- **LangGraph** for DAG node coordination and memory handling.
- **OpenAI API (`openai==1.3.8`)** for LLM-based reasoning.
- **Google Colab** as runtime platform.
- **Graphviz** for visualizing the DAG.

---

##  How It Works

### 1. User Input (CLI)
Users provide:
- A debate topic
- Profession of Agent 1 (e.g., Scientist)
- Profession of Agent 2 (e.g., Philosopher)

This dynamic flow ensures agent roles are not hardcoded.

### 2. LangGraph DAG Setup
A custom `DebateState` is defined to carry:
- The current round
- Memory per agent
- Debate history
- Next speaker
- Final verdict

LangGraph nodes are configured:
- `agent_turn`: Generate LLM response
- `update_memory`: Store full transcript
- `judge`: Summarize and declare a winner

Control is routed using a `route()` function that sends flow to the judge after round 8.

### 3. Execution
The DAG is invoked using `.invoke(initial_state)`:
- 8 rounds are executed (turn-by-turn)
- The judge node is triggered on the 9th transition
- All outputs are printed and saved

### 4. Output Files
- `debate_log_<timestamp>.txt`: Full transcript + final verdict
- `debate_dag_<timestamp>.png`: A visual DAG flow

Files can be downloaded in Colab using:
```python
from google.colab import files
files.download(...)
Sample Flow
User Input:


Topic: Should AI be regulated as medicine?
Agent 1: Scientist
Agent 2: Philosopher
Round 1: Scientist argues

Round 2: Philosopher responds

...

Verdict: Judge declares the winner with reasoning

 DAG Architecture
The DAG diagram (auto-generated) contains:


User Input → Agent 1 Turn → Memory Update → Agent 2 Turn → ...
→ After 8 rounds → Judge → Verdict
Graphviz renders this into a .png file to satisfy the deliverable.

 Demo Video
https://www.loom.com/share/8cca0174bfdc4c7ab0dd839a210871ea?sid=3975f92d-8105-4a8d-89c1-3f5c057ad98f""""""